In [1]:
#import pandas as pd
import polars as pl
import numpy as np
from datetime import datetime
import json
import os
import math
import pandas as pd
from sklearn.preprocessing import PowerTransformer, StandardScaler
import time

fase = '02_normaliza'

In [2]:
with open('gen_config.json', 'r') as file:
    gen_config =json.load(file)

In [3]:
folder = gen_config['folder']

#entradas
path_group = gen_config['path_group']
path_prod_stats = gen_config['path_prod_stats']

#salidas
path_norm = gen_config['path_norm']
path_transform_stats = gen_config['path_transform_stats']

print(f"{'COMIENZA':-^100}")
print(f"{fase:-^100}")

----------------------------------------------COMIENZA----------------------------------------------
--------------------------------------------02_normaliza--------------------------------------------


In [4]:
df = pl.read_parquet(f"{folder}/{path_group}")
prod_stats = pl.read_parquet(f'{folder}/{path_prod_stats}')

In [5]:
df_norm = df.join(prod_stats, on=['product_id', 'customer_id'], how='left', coalesce=True)
print(f"Columnas en DF:{df_norm.columns}")
print(df_norm.head())

Columnas en DF:['product_id', 'customer_id', 'periodo', 'tn', 'primer_periodo', 'ultimo_periodo', 'values', 'total_tn', 'min_tn', 'average_tn', 'median_tn', 'std_dev_tn', 'iqr_tn', 'max_tn']
shape: (5, 14)
┌────────────┬───────────┬─────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ product_id ┆ customer_ ┆ periodo ┆ tn        ┆ … ┆ median_tn ┆ std_dev_t ┆ iqr_tn    ┆ max_tn    │
│ ---        ┆ id        ┆ ---     ┆ ---       ┆   ┆ ---       ┆ n         ┆ ---       ┆ ---       │
│ i64        ┆ ---       ┆ i64     ┆ f64       ┆   ┆ f64       ┆ ---       ┆ f64       ┆ f64       │
│            ┆ i64       ┆         ┆           ┆   ┆           ┆ f64       ┆           ┆           │
╞════════════╪═══════════╪═════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 20001      ┆ 10001     ┆ 201701  ┆ 99.43861  ┆ … ┆ 142.01462 ┆ 106.03278 ┆ 140.01875 ┆ 439.90647 │
│            ┆           ┆         ┆           ┆   ┆           ┆ 1         ┆           

In [6]:
# juguete = df_norm.filter((pl.col('product_id') == 20011) & (pl.col('customer_id') == 10001))[['product_id','customer_id', 'periodo','tn']]
# juguete, lambda_, scale_, mean_, var_ = power_transform(juguete)
# juguete = power_inverse_transform(juguete, lambda_, mean_, var_)
# juguete

# ACA SUCEDE LA NORMALIZACION

In [8]:
df_norm = df_norm.with_columns([
    ((pl.col("periodo").cast(pl.Utf8) + "01").str.to_date("%Y%m%d")).alias("periodo_dt"),  
])

primer_periodo = df_norm['periodo_dt'].min()

df_norm = df_norm.with_columns(
    ((pl.col('periodo_dt').dt.year() - primer_periodo.year) * 12 +
    (pl.col('periodo_dt').dt.month() - primer_periodo.month)).alias('mes_indice')
)

df_pivot = df_norm.pivot(values='tn', index=['product_id', 'customer_id'], columns='mes_indice')
df_pivot = df_pivot.to_pandas()


In [9]:
def power_transform(row_df):
    pt = PowerTransformer()
    transformed_data = pt.fit_transform(row_df[2:].to_numpy().reshape(-1, 1)).flatten()
    #group_df = group_df.with_columns(pl.Series('tn_trans', transformed_data))

    stdscaler = pt.__getattribute__('_scaler')
    
    lambda_ = pt.lambdas_[0]
    scale_ = stdscaler.scale_[0]
    mean_ = stdscaler.mean_[0]
    var_ = stdscaler.var_[0]

    return transformed_data, lambda_, scale_, mean_, var_

In [40]:
def power_inverse_transform(x_trans, lambda_, mean_, var_):
    try:
        x = x_trans.to_numpy().reshape(-1, 1)
    except:
        x = np.array(x_trans)

    x = x * var_ ** 0.5 + mean_

    x_inv = np.zeros_like(x)
    pos = x >= 0

    # when x >= 0
    if abs(lambda_) < np.spacing(1.0):
        x_inv[pos] = np.exp(x[pos]) - 1
    else:  # lambda_ != 0
        x_inv[pos] = np.power(x[pos] * lambda_ + 1, 1 / lambda_) - 1

    # when x < 0
    if abs(lambda_ - 2) > np.spacing(1.0):
        x_inv[~pos] = 1 - np.power(-(2 - lambda_) * x[~pos] + 1, 1 / (2 - lambda_))
    else:  # lambda_ == 2
        x_inv[~pos] = 1 - np.exp(-x[~pos])

    x_orig = x_inv.flatten()

    return x_orig

In [11]:
import warnings

warnings.filterwarnings('ignore')

In [12]:
transform_stats = pd.DataFrame(columns=['product_id', 'customer_id', 'pwr_lambda', 'pwr_scale', 'pwr_mean', 'pwr_var'])
df_pwr_pt = pd.DataFrame(columns=['product_id', 'customer_id', 'mes_indice', 'tn', 'tn_trans'])
df_pwr = pd.DataFrame(columns=['product_id', 'customer_id', 'mes_indice', 'tn', 'tn_trans'])
meses = pd.Series(df_pivot.columns[2:])

start_time = time.time()

for i, row in df_pivot.iterrows():
    if i % 500 == 0:
        print(f"Transformando serie {i} de {df_pivot.shape[0]}, {round(i/df_pivot.shape[0]*100, 2)}%")
        df_pwr = pd.concat([df_pwr, df_pwr_pt], ignore_index=True, axis=0)  
        df_pwr_pt = pd.DataFrame(columns=['product_id', 'customer_id', 'mes_indice', 'tn', 'tn_trans'])
        print(f"Acumulado: {df_pwr.shape}, {round(time.time() - start_time, 2)} segundos")
        start_time = time.time()
    
    x_trans, lambda_, scale_, mean_, var_ = power_transform(row)
    df_row = pd.DataFrame({'product_id' : row.iloc[0],
                            'customer_id' : row.iloc[1],
                            'mes_indice' : meses.values,
                            'tn' : row.iloc[2:],
                            'tn_trans' : x_trans})
    df_pwr_pt = pd.concat([df_pwr_pt, df_row], ignore_index=True, axis=0)    
    
    transform_stats_row = pd.DataFrame({'product_id': row.iloc[0],
                        'customer_id': row.iloc[1],
                        'pwr_lambda': lambda_,
                        'pwr_scale': scale_,
                        'pwr_mean': mean_,
                        'pwr_var': var_
                        }, index=[i])
    transform_stats = pd.concat([transform_stats, transform_stats_row], ignore_index=True, axis=0)

#resto
print(f"Transformando serie {i} de {df_pivot.shape[0]}, {round(i/df_pivot.shape[0]*100, 2)}%")
df_pwr = pd.concat([df_pwr, df_pwr_pt], ignore_index=True, axis=0)  
df_pwr_pt = pd.DataFrame(columns=['product_id', 'customer_id', 'mes_indice', 'tn', 'tn_trans'])
print(f"Acumulado: {df_pwr.shape}, {round(time.time() - start_time, 2)} segundos")


# para controlar
# a = df_pwr[(df_pwr['product_id'] == 20001) & (df_pwr['customer_id'] == 10001)]['tn_trans']
# b = power_inverse_transform(a,
#                         lambda_=transform_stats.iloc[0]['pwr_lambda'],
#                         mean_=transform_stats.iloc[0]['pwr_mean'],
#                         var_=transform_stats.iloc[0]['pwr_var'])
# c = df_pwr[(df_pwr['product_id'] == 20001) & (df_pwr['customer_id'] == 10001)]['tn']

# pd.DataFrame({'a': a, 'b': b, 'c': c})

Transformando serie 0 de 262805, 0.0%
Acumulado: (0, 5), 0.03 segundos
Transformando serie 500 de 262805, 0.19%
Acumulado: (18000, 5), 0.92 segundos
Transformando serie 1000 de 262805, 0.38%
Acumulado: (36000, 5), 0.89 segundos
Transformando serie 1500 de 262805, 0.57%
Acumulado: (54000, 5), 0.87 segundos
Transformando serie 2000 de 262805, 0.76%
Acumulado: (72000, 5), 0.89 segundos
Transformando serie 2500 de 262805, 0.95%
Acumulado: (90000, 5), 0.88 segundos
Transformando serie 3000 de 262805, 1.14%
Acumulado: (108000, 5), 0.91 segundos
Transformando serie 3500 de 262805, 1.33%
Acumulado: (126000, 5), 0.9 segundos
Transformando serie 4000 de 262805, 1.52%
Acumulado: (144000, 5), 0.91 segundos
Transformando serie 4500 de 262805, 1.71%
Acumulado: (162000, 5), 0.91 segundos
Transformando serie 5000 de 262805, 1.9%
Acumulado: (180000, 5), 0.9 segundos
Transformando serie 5500 de 262805, 2.09%
Acumulado: (198000, 5), 0.9 segundos
Transformando serie 6000 de 262805, 2.28%
Acumulado: (21600

In [13]:
df_pwr = pl.from_pandas(df_pwr)
transform_stats = pl.from_pandas(transform_stats)

In [14]:
df_pwr = df_pwr.with_columns([
    pl.col('product_id').cast(pl.Int64),
    pl.col('customer_id').cast(pl.Int64),
    pl.col('mes_indice').cast(pl.Int32),
])

transform_stats = transform_stats.with_columns([
    pl.col('product_id').cast(pl.Int64),
    pl.col('customer_id').cast(pl.Int64),
])

In [15]:
df_pwr.write_parquet(f'{folder}/pwr.parquet')
transform_stats.write_parquet(f'{folder}/{path_transform_stats}')

In [16]:
df_pwr = pl.read_parquet(f'{folder}/pwr.parquet')
transform_stats = pl.read_parquet(f'{folder}/{path_transform_stats}')

In [17]:
df_norm = df_norm.join(df_pwr[['product_id', 'customer_id', 'mes_indice', 'tn_trans']],
            on=['product_id', 'customer_id', 'mes_indice'], how='left', coalesce=True)
df_norm = df_norm.with_columns(pl.col('tn_trans').alias('tn_norm'))

In [18]:
#df_temp = df_norm.join(transform_stats, on=['product_id', 'customer_id'], how='left', coalesce=True)

In [25]:
df_norm = df_norm.with_columns([
    ((pl.col('tn') - pl.col('average_tn')) / pl.col('std_dev_tn')).alias('tn_standard'),
])

In [26]:
df_norm = df_norm.sort(by=['product_id', 'customer_id', 'periodo'])

In [27]:
df_norm.write_parquet(f'{folder}/{path_norm}')
transform_stats.write_parquet(f'{folder}/{path_transform_stats}')

In [28]:
print(f"df_norm completo: {df_norm.shape}")
print(f"df_norm diciembre: {df_norm.filter(pl.col('periodo') == 201912).shape}")

df_norm completo: (8042749, 19)
df_norm diciembre: (262805, 19)


In [32]:
# df_temp = df_norm.join(transform_stats, on=['product_id', 'customer_id'], how='left', coalesce=True)

In [36]:
# df_temp = df_temp.to_pandas()

In [74]:
# df_temp['tn_inversa'] = df_temp.apply(lambda row: power_inverse_transform(row['tn_trans'], row['pwr_lambda'], row['pwr_mean'], row['pwr_var']), axis=1)

In [75]:
# df_temp['tn_inversa'] = [tn_inv[0] for tn_inv in df_temp['tn_inversa']]

In [76]:
# test = df_temp['tn_inversa'] - df_temp['tn']

In [77]:
# test = df_temp[np.abs(df_temp['tn_inversa'] - df_temp['tn']) >= 0.01]
# test

In [57]:
# df_temp.to_parquet(f'{folder}/temp.parquet', index=False)

In [30]:
print(f"{fase:-^100}")
print(f"{'FINALIZA':-^100}\n\n\n")

--------------------------------------------02_normaliza--------------------------------------------
----------------------------------------------FINALIZA----------------------------------------------



